## 인사혁신처 소청사례
* https://sochung.mpm.go.kr/home/user/board_list.do?PAGE_FLAG=EXSR&PAGE_MODE=LIST

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

## 크롤링할 URL

In [2]:
url = "https://sochung.mpm.go.kr/home/user/board_list.do?PAGE_FLAG=EXSR&PAGE_MODE=LIST"
params = "&PAGE_INT=8&PAGE_SIZE=10&searchField=4&PAGE_BIDX=&PAGE_TMP3=&PAGE_TMP4=&PAGE_TMP5=&show_Flag=Y&searchWord=&PAGE_TMP1=&PAGE_SDATE=2010-01-01&PAGE_EDATE=2020-07-17&PAGE_MIDX=&PAGE_TMP2="

## 파라메터값 변경을 위해 dict 형태로 변경하기

In [3]:
parameter = {}
for p in params.split("&")[1:]:
    key = p.split("=")[0]
    val = p.split("=")[1]
    parameter[key] = val

parameter

{'PAGE_INT': '8',
 'PAGE_SIZE': '10',
 'searchField': '4',
 'PAGE_BIDX': '',
 'PAGE_TMP3': '',
 'PAGE_TMP4': '',
 'PAGE_TMP5': '',
 'show_Flag': 'Y',
 'searchWord': '',
 'PAGE_TMP1': '',
 'PAGE_SDATE': '2010-01-01',
 'PAGE_EDATE': '2020-07-17',
 'PAGE_MIDX': '',
 'PAGE_TMP2': ''}

## 전기간 검색을 위해 검색기간 초기화 하기

In [ ]:
parameter["PAGE_SDATE"] = ''
parameter["PAGE_EDATE"] = ''
parameter

## 상세내용 수집을 위한 링크 idx 수집

In [51]:
def get_href_list(response):
    html = bs(response.text)
    href_list = []
    a_tag = html.select("table.board-list > tbody > tr > td > a")
    for a in a_tag:
        href_idx = a["href"].split("IDX=")[1].split("&")[0]
        href_list.append(href_idx)
    return href_list

## 데이터 수집

In [15]:
import time 
import random
page_list = []
href_idx_list = []
total_page = 497
# total_page = 5
for page_no in range(493, total_page+1):
    time.sleep(random.randint(1, 2))
    parameter["PAGE_INT"] = page_no
    response = requests.post(url, parameter)
    if response.status_code == 200:
        table = pd.read_html(response.text)
        page_list.extend(table[1].values.tolist())
        # 링크 idx 수집
        idx_list = get_href_list(response)
        href_idx_list.extend(idx_list)
        if page_no % 10 == 0 :
            print(page_no)

### IDX 목록저장

In [62]:
df_idx = pd.DataFrame(href_idx_list, columns=["idx"])
df_idx.to_csv("sochung_idx.csv", index=False)

In [63]:
pd.read_csv("sochung_idx.csv").head()

,idx
0,5719868
1,5719539
2,5719526
3,5719523
4,5719518


### 소청목록 저장

In [16]:
page_list

[[19, 19980403, '해임', '지시명령위반', '기각', '1998-08-21'],
 [18, 19980402, '해임', '허위보고 및 지시', '감경', '1998-08-21'],
 [17, 19980401, '해임', '근무태만', '기각', '1998-08-21'],
 [16, 19980369, '견책', '직무태만(업무처리 소홀)', '취소', '1998-08-12'],
 [15, 19980356, '해임', '회계관련 물의', '기각', '1998-07-22'],
 [14, 19980333, '견책', '직무태만(업무처리 소홀)', '기각', '1998-07-15'],
 [13, 19980296, '견책', '직무태만(업무처리 소홀)', '기각', '1998-07-01'],
 [12, 19980294, '견책', '지시명령위반', '기각', '1998-07-08'],
 [11, 19980268, '정직3월', '회계관련 물의', '감경', '1998-06-22'],
 [10, 19980247, '견책', '근무태만', '기각', '1998-06-10'],
 [9, 19980228, '해임', '직무유기', '감경', '1998-06-03'],
 [8, 19980216, '해임', '근무태만', '감경', '1998-06-03'],
 [7, 19980166, '감봉1월', '허위보고 및 지시', '기각', '1998-05-01'],
 [6, 19980151, '해임', '도박행위', '감경', '1998-04-22'],
 [5, 19980143, '감봉2월', '사건묵살', '기각', '1998-04-24'],
 [4, 19980134, '직권면직', '근무태만', '기각', '1998-04-15'],
 [3, 19980100, '해임', '지시명령위반', '기각', '1998-04-08'],
 [2, 19980059, '견책', '근무태만', '기각', '1998-03-18'],
 [1, 19980001, '감봉3월', '직무태만(업무처리

In [12]:
df = pd.DataFrame(page_list, columns=['번호', '사건번호', '원처분', '비위유형', '결정유형', '결정일자'])
print(df.shape)
df.head()

(0, 6)


,번호,사건번호,원처분,비위유형,결정유형,결정일자


In [7]:
df.tail()

,번호,사건번호,원처분,비위유형,결정유형,결정일자
4934,5,19980143,감봉2월,사건묵살,기각,1998-04-24
4935,4,19980134,직권면직,근무태만,기각,1998-04-15
4936,3,19980100,해임,지시명령위반,기각,1998-04-08
4937,2,19980059,견책,근무태만,기각,1998-03-18
4938,1,19980001,감봉3월,직무태만(업무처리 소홀),기각,1998-02-27


In [8]:
df.to_csv("sochung_all.csv", index=False)

In [9]:
pd.read_csv("sochung_all.csv")

,번호,사건번호,원처분,비위유형,결정유형,결정일자
0,4939,20190660,해임,금품수수(향응수수),감경,2020-01-16
1,4938,20190737,견책,품위손상,기각,2020-02-11
2,4937,20190718,견책,품위손상,감경,2020-02-04
3,4936,20190665,감봉1월,품위손상,감경,2020-01-09
4,4935,20190668,직권면직,기타,기각,2020-02-20
...,...,...,...,...,...,...
4934,5,19980143,감봉2월,사건묵살,기각,1998-04-24
4935,4,19980134,직권면직,근무태만,기각,1998-04-15
4936,3,19980100,해임,지시명령위반,기각,1998-04-08
4937,2,19980059,견책,근무태만,기각,1998-03-18
